In [33]:
import pandas as pd
# df = pd.read_csv('90d.csv')
df = pd.read_csv('d2.csv')
k = 0.5
losscut = 200
profitcut = 200
fee_rate_pct = 0.075
slippage_pct = 0.2

df['range'] = (df['high'] - df['low']) * k
df['range_shift1'] = df['range'].shift(1)
df['long_target'] = df['open'] + df['range_shift1']
df['short_target'] = df['open'] - df['range_shift1']
df['close-open'] = df['close'] - df['open']

# 양봉인지 여부.
df['is_green'] = df['close-open'] >= 0

In [34]:
def long_sell_price(x):
    if x['is_green']:
        # 오름봉이면, 최대값에서 100불이 빠졌을때 익절한다.
        return x['high'] - profitcut if x['close'] <= x['high'] - profitcut else x['close']
    else:
        # 내린봉이면. low 가 진입가보다 -30불이상 더 낮으면 손절한다.
        return x['long_target'] - losscut if x['low'] <= x['long_target'] - losscut else x['close']


def short_sell_price(x):
    if not x['is_green']:
        # 내림봉이면 익절, 최대값에서 100불이 빠졌을때 익절한다.
        return x['low'] + profitcut if x['close'] >= x['low'] + profitcut else x['close']
    else:
        # 오름봉이면 손절. low 가 진입가보다 -30불이상 더 낮으면 손절한다.
        return x['short_target'] + losscut if x['high'] >= x['short_target'] + losscut else x['close']

In [35]:
import numpy as np
# 롱타겟
# df['sell_price_long'] = df['close'] 
df['sell_price_long'] = df.apply(lambda x: long_sell_price(x), axis=1)
df['pnl_long'] = np.where(df['high'] > df['long_target'], df['sell_price_long'] - df['long_target'], 0)
df['profit_long%'] = df['pnl_long'] / df['long_target']
df['ror_long'] = np.where(df['high'] > df['long_target'], df['sell_price_long'] / df['long_target'], 1)

# 숏타켓
# df['sell_price_short'] = df['close'] 
df['sell_price_short'] = df.apply(lambda x: short_sell_price(x), axis=1)
df['pnl_short'] = np.where(df['low'] < df['short_target'], df['short_target'] - df['sell_price_short'], 0)
df['profit_short%'] = df['pnl_short'] / df['short_target']
df['ror_short'] = np.where(df['low'] < df['short_target'], df['short_target'] / df['sell_price_short'], 1)

In [36]:
# 1보다 크면 수익있는것.
df['long_win'] = np.where(df['ror_long'] > 1, 1, 0)
df['long_lose'] = np.where(df['ror_long'] < 1, 1, 0)
df['short_win'] = np.where(df['ror_short'] > 1, 1, 0)
df['short_lose'] = np.where(df['ror_short'] < 1, 1, 0)
df['cum_long_win'] = df['long_win'].cumsum()
df['cum_long_lose'] = df['long_lose'].cumsum()
df['cum_short_win'] = df['short_win'].cumsum()
df['cum_short_lose'] = df['short_lose'].cumsum()
df['cumprod_long'] = df['ror_long'].cumprod()
df['cumprod_short'] = df['ror_short'].cumprod()
# df['hpr'] = df['ror_long'].cumprod()
# df['cumprod'] = df['ror_long'].cumprod()
df['dd_long'] = (df['cumprod_long'].cummax() - df['cumprod_long']) / df['cumprod_long'].cummax() * 100
df['dd_short'] = (df['cumprod_short'].cummax() - df['cumprod_short']) / df['cumprod_short'].cummax() * 100

last = df.iloc[-1, :]
trade_count = last['cum_long_win'] + last['cum_long_lose'] + last['cum_short_win'] + last['cum_short_lose']

fee = trade_count * 2 * fee_rate_pct
slippage = trade_count * 2 * slippage_pct

# print(df.to_string())
print(df)
print('-----------------------')
print(last['cum_long_win'], last['cum_long_lose'], last['cum_short_win'], last['cum_short_lose'])
print('trade_count : ', trade_count)
print('fee%: ', fee)
print('slippage%: ', slippage)
print("MDD_long: ", df['dd_long'].max())
print("MDD_short: ", df['dd_short'].max())

     Unnamed: 0      open_time      open      high       low     close  \
0             0  1516233600000  10972.59  11878.82  10435.33  10961.97   
1             1  1516320000000  10960.00  11795.00  10360.00  11474.98   
2             2  1516406400000  11474.98  13099.00  11412.45  12799.94   
3             3  1516492800000  12799.80  12799.80  10965.00  11530.00   
4             4  1516579200000  11530.00  11926.35   9900.24  10760.05   
..          ...            ...       ...       ...       ...       ...   
705         705  1577145600000   7317.30   7436.68   7157.04   7255.77   
706         706  1577232000000   7255.77   7271.77   7128.86   7204.63   
707         707  1577318400000   7205.01   7435.00   7157.12   7202.00   
708         708  1577404800000   7202.00   7275.86   7076.42   7254.74   
709         709  1577491200000   7254.77   7360.00   7238.67   7339.13   

           volume     close_time      volume.1  trades  ...  short_win  \
0    48464.434937  1516319999999  5.4